In [1]:
%run Deck.ipynb
%run Player.ipynb

In [2]:
import random
from copy import copy, deepcopy
import numpy as np

In [3]:
class Board:
    
    def __init__(self, deck :list):
        
        self.deck = deck
        self.player1, self.player2, self.player3 = Player("player1"), Player("player2"), Player("player3")
        self.player_list = [self.player1, self.player2, self.player3]
        self.player_order = copy(self.player_list)
        
        self.number_cards_in_dog = 6 
        self.number_cards_played = 0

        self.dog = []
        self.known_cards = []

        self._distr_cards() # distr the cards to the player1 and to the dog
        
        self.cards_on_table = []
        self.possible_state = True
        self.legals_actions = self.get_legal_actions()
    
    def _distr_cards(self):
        """Distribute the cards when the board is initialized"""
        
        random_sample = random.sample(range(0, len(deck)), 30)
        
        self.dog = [self.deck[i] for i in random_sample[:self.number_cards_in_dog]]
        self.player1.hand = [self.deck[i] for i in random_sample[self.number_cards_in_dog:]]
        
        self.known_cards = set(self.dog + self.player1.hand)
        
    
    def get_legal_actions(self):
        """return the list of playable actions"""
        
        player_to_play = self.player_order[0]
        #print("name joueur", player_to_play.name)
        
        self.player1 = [player for player in self.player_list if player.name == "player1"][0]
        
        # If it's J1 turn to play: look at his hand and at the cards on the table.
        if player_to_play.name == "player1":
            hand = player_to_play.hand  
            
            #print("Player1 Turn")
            # Then we select possible card, based on what is arleady on the table and player1 hand
            if self.cards_on_table == []: 
                return hand

            first_played = self.cards_on_table[0]
            excuse = [card for card in hand if card.color == "Joker"] # [] if J1 doesn't have the excuse

            # If the first ward played is the excuse we look at the second 
            if first_played.color == "Joker":
                if len(self.cards_on_table) > 1:
                    first_played = self.cards_on_table[1]

                # if there isn't other played card, we can play anything
                else:
                    return hand

            # if the first card is a trump, we need to play an higher one else we play something else
            if first_played.color == "trump":
                higher_trump = [card for card in hand if card.color == "trump" and card.value > self.cards_on_table[-1].value]  
                if higher_trump:
                    return higher_trump + excuse

                trumps = [card for card in hand if card.color == "trump"]
                if trumps:
                    return trumps + excuse
                else:
                    return hand

            first_color = [card for card in hand if card.color == first_played.color]

            if first_color:
                return first_color + excuse    

            trumps = [card for card in hand if card.color == "trump"]  
            if trumps:
                return trumps + excuse

            else:
                return hand

        # If it's not the turn of J1, we simulate a card based on what J1 know
        else:
            #print(f"Simulate {player_to_play.name} plays")
            
            # Unknown card card are thoses not in history or player1 hand
            self.unknown_cards = [card for card in self.deck + list(self.known_cards) if card not in self.deck or card not in self.known_cards]
            # might add card that have the same name but different reference
            
            possible_to_play = self.unknown_cards
            
            # Then we return the possibles card, based on what has been played before
            if not player_to_play.has_hearts:
                possible_to_play = [card for card in possible_to_play if card.color !="heart"]
                
            if not player_to_play.has_spades:
                possible_to_play = [card for card in possible_to_play if card.color !="spade"]
                
            if not player_to_play.has_diamonds:
                possible_to_play = [card for card in possible_to_play if card.color !="diamond"]
                
            if not player_to_play.has_clubs:
                possible_to_play = [card for card in possible_to_play if card.color !="club"]
                
            if not player_to_play.has_trumps:
                possible_to_play = [card for card in possible_to_play if card.color !="trump"]
            
            possible_to_play = [card for card in possible_to_play if card.value < player_to_play.max_trump_playable]
                       
            if len(possible_to_play) + 1 < len(self.player1.hand):
                self.possible_state = False
                #print("Impossible state, not enough card to simulate the end of the game")
            return possible_to_play
    
    
    def move(self, action):
        """Return the next state of the game"""
        
        assert self.possible_state, "Impossible, or useless to continue expanding this node" 
        
        current_player = self.player_list[3 - len(self.player_order)]
        #print("current_player", current_player.name)
        
        cls = self.__class__
        next_state = cls.__new__(cls)
        
        # not supposed to be copied 
        next_state.deck = self.deck
        next_state.dog = self.dog 
        next_state.possible_state = self.possible_state # it's a boolean
        
        # Copy the list but not the element inside it
        next_state.known_cards = copy(self.known_cards)
        next_state.cards_on_table = copy(self.cards_on_table)
        next_state.player_order = copy(self.player_order)
        
        # Copy the players state, otherwise when changing a player infos, it'd also change it on the parent node.
        next_state.player_list = []
        
        for player in self.player_list:
            
            cls_player = player.__class__
            next_player_state = cls.__new__(cls_player)
            
            # Copy the list but not the element inside it
            next_player_state.hand = copy(player.hand) 
            
            # don't need to copy immutable object
            next_player_state.name = player.name 
            next_player_state.actual_score = player.actual_score
            next_player_state.has_hearts = player.has_hearts
            next_player_state.has_spades = player.has_spades
            next_player_state.has_diamonds = player.has_diamonds
            next_player_state.has_clubs = player.has_clubs
            next_player_state.has_trumps = player.has_trumps
            next_player_state.max_trump_playable = player.max_trump_playable
            next_player_state.number_card_played = player.number_card_played + 1
            
            next_state.player_list.append(next_player_state)
        
        next_player = next_state.player_list[3 - len(self.player_order)]
        
        # if current player is j1, remove the played card from his hand in the next state
        if current_player.name == "player1": #next_player
             next_player.hand.remove(action)
        # Based on what the current player played, remove him the cards he'll no more be able to play
        # For example if heart is asked and he play trump, it means he won't be able to play heart for the rest of the game.
        next_state.remove_player_permission(next_player, action)
               
        # add card on card on table 
        next_state.cards_on_table.append(action)
        #print([card.name for card in next_state.cards_on_table])
        
        # known_cards is a set, so we can still add the cards played by player1, it won't change anything
        next_state.known_cards.add(action)
        
        next_state.number_cards_played = self.number_cards_played + 1
        
        # change the player order 
        if len(self.player_order) > 1:
            next_state.player_order.pop(0)
        
        else:
            win_index = next_state.win_manche()
            next_state.player_list = next_state.player_list[win_index:] + next_state.player_list[:win_index]
            next_state.player_order = copy(next_state.player_list) 
            next_state.cards_on_table = []
        
        # add the legal actions, and change if it's a terminal node or not.
        next_state.legals_actions = next_state.get_legal_actions()
                
        return next_state
    
    
    def remove_player_permission(self, player, action):
        """Based on what the current player played, remove him the cards he'll no more be able to play"""
        
        if self.cards_on_table != []:
            first_color = self.cards_on_table[0].color
            excuse = [card for card in self.deck if card.color == "Joker"]

            # if the first card played if the excuse, we look at the next one
            if first_color == "Joker" and len(self.cards_on_table) > 1:
                first_color = self.cards_on_table[1].color
            
            # if the player play a lower trump than the highest on the table, the highest is it's boundary
            if first_color == "trump" and action.color == "trump":
                highest_trump = max([card.value for card in self.cards_on_table if card.color=="trump"])
                
                if action.value < highest_trump:
                    player.max_trump_playable = highest_trump
                    #print(f"{player.name} can no more play trump higher than {highest_trump}")

            # Check for possibles plyables cards 
            if first_color == "heart" and action.color not in ("heart", "Joker"):
                player.has_hearts = False
                #print(f"{player.name} can no more play heart")
                
                if action.color != "trump":
                    player.has_trumps = False
                    #print(f"{player.name} can no more play trumps")
                    
            if first_color == "spade" and action.color not in ("spade", "Joker"):
                player.has_spades = False
                #print(f"{player.name} can no more play spade")
                
                if action.color != "trump":
                    player.has_trumps = False
                    #print(f"{player.name} can no more play trumps")
                    
            if first_color == "club" and action.color not in ("club", "Joker"):
                player.has_clubs = False
                #print(f"{player.name} can no more play club")
                
                if action.color != "trump":
                    player.has_trumps = False
                    #print(f"{player.name} can no more play trumps")
                    
            if first_color == "diamond" and action.color not in ("diamond", "Joker"):
                player.has_diamonds = False
                #print(f"{player.name} can no more play diamonds")
                
                if action.color != "trump":
                    player.has_trumps = False
                    #print(f"{player.name} can no more play trumps")
            
            if first_color == "trump" and action.color not in ("trump", "Joker"):
                player.has_trumps = False
                #print(f"{player.name} can no more play trumps")

    
    def win_manche(self):
        """return the index of the card that won the tricks, i.e 0, 1 or 2"""
        
        first_played = self.cards_on_table[0]
        if first_played.color == "Joker":
            first_played = self.cards_on_table[1]

        # If no trumps are played, the strongest card of the asked color win the trick
        if "trump" not in [card.color for card in self.cards_on_table]:
            win_index = np.argmax([card.value if card.color==first_played.color else 0 for card in self.cards_on_table])
            
            # We count J1 player's points
            if self.player_list[win_index].name == "player1":
                self.player_list[win_index].actual_score += sum([card.points for card in self.cards_on_table])
                
            return win_index 

        # If one or more trumps have been played, the highest win the trick 
        else : 
            win_index = np.argmax([card.value if card.color=='trump' else 0 for card in self.cards_on_table])          
           
            # We count J1 player's points
            if self.player_list[win_index].name == "player1":
                self.player_list[win_index].actual_score += sum([card.points for card in self.cards_on_table])
                
            return win_index
        
    def is_not_terminal_node(self):
        """return if a node is terminal, ie if it should not be expanded. For example when the current game can't be finished
        or when the game is finished and all player have played their 24 cards"""

        if self.possible_state:
            return True
        
        else:
            return False
        

        
    def game_result(self):
        """return the result of the game, 0 if it didn't finished, 1 if j1 won, else -1"""
        score_to_win = 36
        
        if self.number_cards_played >= 72:
            print("erreur pas possible de jouer plus de 72 cartes ")
            return self
        # If 72 card (78 - 6 in the dog) have been played = game finished correctly
        
        if self.number_cards_played == 72:
            if self.player1.actual_score > score_to_win:
                #print("Joueur 1 gagne")
                return 1
            else:
                #print("Joueur 1 perd")
                return -1
        else: 
            return 0
